## 2 Whisper Baseline

Before fine-tuning our own model, we will first decode with the whisper model to get a baseline word error rate.

Imports and configuration

In [1]:
import os
import torch
import whisper
import pandas as pd
import numpy as np
from tqdm import tqdm
from jiwer import wer, cer
from bs4 import BeautifulSoup
import re
from pathlib import Path
import warnings
import setproctitle
warnings.filterwarnings("ignore")

PROCESS_NAME = "romansh-asr"
setproctitle.setproctitle(PROCESS_NAME)

DATA_PATH = "romansh-data/sursilvan-mini/"
WHISPER_MODEL = "medium"
BATCH_SIZE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_FILE = "whisper_baseline_results.txt"

Checking GPU availability

In [2]:
print("="*60)
print("Whisper Baseline for Romansh ASR")
print("="*60)
print(f"Model: {WHISPER_MODEL}")
print(f"Device: {DEVICE}")
print(f"Data path: {DATA_PATH}")
print("="*60)

if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("CUDA not available - running on CPU (this will be slow!)")

Whisper Baseline for Romansh ASR
Model: medium
Device: cuda
Data path: romansh-data/sursilvan-mini/
GPU: NVIDIA GeForce RTX 3090
GPU Memory: 25.30 GB


Defining some helpful helper functions

In [7]:
def clean_html(text):
    """Remove HTML tags from text"""
    if pd.isna(text) or not isinstance(text, str):
        return ""
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def load_data(tsv_path):
    """Load and prepare data from TSV file"""
    print(f"Loading data from {tsv_path}")
    df = pd.read_csv(tsv_path, sep='\t')
    
    print("Cleaning HTML from sentences...")
    df['clean_sentence'] = df['sentence'].apply(clean_html)
    
    print("\nSample cleaning results:")
    for i in range(min(3, len(df))):
        print(f"Original: {df['sentence'].iloc[i][:100]}...")
        print(f"Cleaned:  {df['clean_sentence'].iloc[i][:100]}...")
        print()
    
    return df

def calculate_metrics(references, hypotheses):
    """Calculate WER and CER with better error handling"""
    # Filter out empty or invalid pairs
    valid_pairs = []
    for ref, hyp in zip(references, hypotheses):
        # Skip if reference is empty or just whitespace
        if not isinstance(ref, str) or not ref.strip():
            continue
        # Skip if hypothesis is empty (but still count these as errors if reference exists)
        if not isinstance(hyp, str):
            hyp = ""
        
        valid_pairs.append((ref.strip(), hyp.strip()))
    
    if not valid_pairs:
        print("Warning: No valid reference-hypothesis pairs found")
        return None, None
    
    refs, hyps = zip(*valid_pairs)
    
    try:
        wer_score = wer(refs, hyps)
        cer_score = cer(refs, hyps)
        return wer_score, cer_score
    except Exception as e:
        print(f"Warning: Error calculating metrics: {e}")
        print(f"First few references: {refs[:3]}")
        print(f"First few hypotheses: {hyps[:3]}")
        return None, None

Loading the whisper model

In [8]:
print(f"\nLoading Whisper {WHISPER_MODEL} model...")
model = whisper.load_model(WHISPER_MODEL, device=DEVICE)
print("Model loaded successfully!")


Loading Whisper medium model...


Model loaded successfully!


Decoding with the whisper model

In [9]:
def transcribe_all_with_language(model, audio_paths):
    """Transcribe all audio files with a single progress bar"""
    transcriptions = []
    detected_languages = []
    
    # Single progress bar for all files
    for audio_path in tqdm(audio_paths, desc="Transcribing audio files"):
        try:
            if not os.path.exists(audio_path):
                transcriptions.append("")
                detected_languages.append("missing")
                continue
            
            result = model.transcribe(
                audio_path,
                task="transcribe",
                fp16=torch.cuda.is_available(),
                verbose=False
            )
            
            transcriptions.append(result['text'].strip())
            detected_languages.append(result.get('language', 'unknown'))
            
        except Exception as e:
            transcriptions.append("")
            detected_languages.append("error")
    
    return transcriptions, detected_languages

Processing all splits

In [12]:
splits = ['train', 'validated', 'test']
all_results = {}
language_summary = {}

for split in splits:
    print(f"\n{'='*50}")
    print(f"Processing {split} split")
    print(f"{'='*50}")
    
    tsv_path = os.path.join(DATA_PATH, f"{split}.tsv")
    clips_path = os.path.join(DATA_PATH, "clips")
    
    if not os.path.exists(tsv_path):
        print(f"{split}.tsv not found, skipping...")
        continue
    
    df = load_data(tsv_path)
    
    audio_paths = [os.path.join(clips_path, path) for path in df['path']]
    
    existing_indices = [i for i, path in enumerate(audio_paths) if os.path.exists(path)]
    missing_count = len(audio_paths) - len(existing_indices)
    
    if missing_count > 0:
        print(f"{missing_count} audio files missing, filtering them out")
        df = df.iloc[existing_indices].reset_index(drop=True)
        audio_paths = [audio_paths[i] for i in existing_indices]
    
    print(f"Total utterances: {len(df)}")
    
    print(f"\nRunning Whisper transcription...")
    hypotheses, detected_langs = transcribe_all_with_language(model, audio_paths)
    
    df['detected_language'] = detected_langs
    
    references = df['clean_sentence'].tolist()
    wer_score, cer_score = calculate_metrics(references, hypotheses)
    
    all_results[split] = {
        'wer': wer_score,
        'cer': cer_score,
        'count': len(df),
        'references': references[:5],
        'hypotheses': hypotheses[:5],
        'languages': detected_langs[:5]
    }
    
    lang_counts = df['detected_language'].value_counts()
    language_summary[split] = lang_counts
    
    print(f"\nResults for {split}:")
    print(f"   Utterances: {len(df)}")
    if wer_score is not None:
        print(f"   WER: {wer_score:.4f} ({wer_score*100:.2f}%)")
        print(f"   CER: {cer_score:.4f} ({cer_score*100:.2f}%)")
    else:
        print("   No valid reference-hypothesis pairs")
    
    print(f"\nLanguage detection distribution:")
    for lang, count in lang_counts.items():
        print(f"   {lang}: {count} ({count/len(df)*100:.1f}%)")


Processing train split
Loading data from romansh-data/sursilvan-mini/train.tsv
Cleaning HTML from sentences...

Sample cleaning results:
Original: <p><span b="2442.334" e="2442.484" s="100" data-index="0" class>Il</span> <span b="2442.484" e="2442...
Cleaned:  Il davos temps vegn bia discutau dalla rolla dalla dunna ella baselgia catolica, co vesis vus quella...

Original: <p><span b="806.922" e="807.322" s="83" data-index="0" class>Suenter</span> <span b="807.352" e="807...
Cleaned:  Suenter ina prelecziun facultativa en dretg da bancas el studi ha quei plaschiu fetg bein a mi. Quei...

Original: <p>E per veser quei stuein nus far in viadi da rodund 80 tochen varga 100 kilometers sut la tiara. <...
Cleaned:  E per veser quei stuein nus far in viadi da rodund 80 tochen varga 100 kilometers sut la tiara. Schi...

Total utterances: 92

Running Whisper transcription...


Transcribing audio files:   0%|          | 0/92 [00:00<?, ?it/s]

Detected language: Slovenian


Transcribing audio files:   1%|          | 1/92 [00:01<02:27,  1.62s/it]

Detected language: German


...

Detected language: Slovenian


Transcribing audio files:  28%|██▊       | 26/92 [01:38<05:05,  4.63s/it]

Detected language: Portuguese


Transcribing audio files:  28%|██▊       | 26/92 [01:40<04:14,  3.86s/it]


KeyboardInterrupt: 

Save results to file

In [ ]:
print(f"\nSaving results to {OUTPUT_FILE}")
with open(OUTPUT_FILE, 'w') as f:
    f.write("="*60 + "\n")
    f.write("Whisper Baseline Results\n")
    f.write(f"Model: {WHISPER_MODEL}\n")
    f.write(f"Date: {pd.Timestamp.now()}\n")
    f.write("="*60 + "\n\n")
    
    for split, results in all_results.items():
        f.write(f"\n{split.upper()} Split:\n")
        f.write(f"  Utterances: {results['count']}\n")
        if results['wer'] is not None:
            f.write(f"  WER: {results['wer']:.4f}\n")
            f.write(f"  CER: {results['cer']:.4f}\n")
        
        f.write("\n  Language Distribution:\n")
        for lang, count in language_summary[split].items():
            f.write(f"    {lang}: {count}\n")
        
        f.write("\n  Example transcriptions:\n")
        for i, (ref, hyp, lang) in enumerate(zip(results['references'], 
                                                results['hypotheses'], 
                                                results['languages'])):
            f.write(f"    Ref {i+1}: {ref}\n")
            f.write(f"    Hyp {i+1}: {hyp}\n")
            f.write(f"    Lang {i+1}: {lang}\n\n")

Show summary

In [ ]:
print("\n" + "="*60)
print("Summary")
print("="*60)

total_utterances = 0
for split in splits:
    if split in all_results:
        results = all_results[split]
        print(f"\n{split.upper()}:")
        print(f"  Utterances: {results['count']}")
        total_utterances += results['count']
        if results['wer'] is not None:
            print(f"  WER: {results['wer']:.4f}")
            print(f"  CER: {results['cer']:.4f}")

print(f"\nTotal utterances processed: {total_utterances}")
print("="*60)
print("Done!")